In [1]:
import time
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.cross_validation import KFold
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score  
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from scipy.stats import itemfreq
from sklearn import neighbors, linear_model
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
np.set_printoptions(precision=3, linewidth=100)

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

os.chdir('/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing')




/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
load_img = cv2.imread('/Users/amikar/Downloads/600u6z4invkz.jpg',cv2.IMREAD_COLOR)
#cv2.imshow('image',load_img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [3]:
def getEntropy(signal):
    lensig=signal.size
    symset=list(set(signal))
    numsym=len(symset)
    probabability_distribution=[np.size(signal[signal==i])/(1.0*lensig) for i in symset]
    entropy=np.sum([p*np.log2(1.0/p) for p in probabability_distribution])
    return entropy

In [4]:
def calculateEntropyNeighbourhood(artwork, neighbourhood):
    image = cv2.imread(artwork)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    colorIm=np.array(image)
    grayIm=np.array(gray_image)
    
    N=neighbourhood
    S=grayIm.shape
    E=np.array(grayIm)
    
    for row in range(S[0]): 
            for col in range (S[1]): 
                    Lx=np.max([0,col-N]) 
                    Ux=np.min([S[1],col+N])
                    Ly=np.max([0,row-N])
                    Uy=np.min([S[0],row+N])
                    # makes region 1-D
                    region=grayIm[Ly:Uy,Lx:Ux].flatten()
                    E[row,col]=getEntropy(region)
    
    average=np.mean(E)
    return average


In [5]:
def getDTM(artwork, neighbourhood):
    image = cv2.imread(artwork)
    image32f = np.float32(image)
    mu    = cv2.blur(image32f,(neighbourhood,neighbourhood))
    mu2   = cv2.blur(cv2.multiply(image32f,image32f), (neighbourhood,neighbourhood))
    sigma = cv2.sqrt( mu2 - cv2.multiply(mu, mu) )
    return np.mean(sigma)

In [6]:
getDTM('/Users/amikar/Downloads/new2.jpg',3)

7.5454607

In [7]:
def get_image_info(test_info, dir):
	if dir == 'test':
		images = list(set(list(test_info.image1.unique()) + list(test_info.image2.unique())))
		info = pd.DataFrame(np.array(images).reshape((-1, 1)), columns = ['filename'])
		#print info
	else:
		info = test_info
	
	info['pixelsx'] = np.nan
	info['pixelsy'] = np.nan
	info['size_bytes'] = np.nan
	#info['entropy'] = np.nan
	info['entropy1'] = np.nan
	info['entropy5'] = np.nan
	info['entropy10'] = np.nan
	info['entropy15'] = np.nan
	info['dtm1'] = np.nan
	info['dtm5'] = np.nan
	info['dtm10'] = np.nan
	info['dtm15'] = np.nan
# 	info['entropy20'] = np.nan
#	info['dtm'] = np.nan

	
	j = 0
	for i in info.index.values:
		j += 1        
		try:
			#print i
			#fil = 'C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Wyeths\\'+dir+'\\'+info.loc[i, 'filename']
			fil = '/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/'+info.loc[i, 'filename']
			print fil
			im = Image.open('/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/'+info.loc[i, 'filename'])
			#print im
			#print im.size
			info.loc[i, 'pixelsx'], info.loc[i, 'pixelsy'] = im.size
			#im = cv2.imread(dir+'/'+info.loc[i, 'new_filename'])
			#info.loc[i, 'pixelsx'], info.loc[i, 'pixelsy'] = im.shape[0:2]
			info.loc[i, 'size_bytes'] = os.path.getsize(info.loc[i, 'filename'])
			#info.loc[i, 'entropy'] = calculateEntropyNeighbourhood(fil, 1)
			#print calculateEntropyNeighbourhood(fil, 1)
#			info.loc[i, 'dtm'] = getDTM(fil)
			info.loc[i, 'entropy1'] = calculateEntropyNeighbourhood(fil, 1)
			info.loc[i, 'entropy5'] = calculateEntropyNeighbourhood(fil, 5)
			info.loc[i, 'entropy10'] = calculateEntropyNeighbourhood(fil, 10)
			info.loc[i, 'entropy15'] = calculateEntropyNeighbourhood(fil, 15)
			info.loc[i, 'dtm1'] = getDTM(fil, 1)
			info.loc[i, 'dtm5'] = getDTM(fil, 5)
			info.loc[i, 'dtm10'] = getDTM(fil, 10)
			info.loc[i, 'dtm15'] = getDTM(fil, 15)
# 			info.loc[i, 'entropy20'] = calculateEntropyNeighbourhood(fil, 20)
		except:
			print dir+'\\'+info.loc[i, 'filename']
		if (j%10 == 0):
			print '',
	info=info.dropna()
	print 'info shape',info.shape
	return info.rename(columns={'filename' : 'new_filename'})

In [8]:
#t = make_pairs(train_info)	
def make_pairs(train_info):
	artists = train_info.artist.unique()

	n = train_info.groupby('artist').size()
	n = (2*n**2).sum() 
	t = pd.DataFrame(np.zeros((n, 4)), columns=['artist1', 'image1', 'artist2', 'image2'])
	i = 0
	j = 0
	for m in artists:
		
		a = train_info[train_info.artist==m][['artist', 'new_filename']].values
		use = train_info[train_info.artist != m].index.values
		np.random.shuffle(use)
		#print a.shape, use.shape
		nm = np.mean([a.shape[0]**2, train_info[train_info.artist != m].shape[0] ])

		use = use[0:nm]
		b = train_info[train_info.artist!=m][['artist', 'new_filename']].ix[use, :].values
		#print nm, use.shape, b.shape
		a2 = pd.DataFrame(np.concatenate([np.repeat(a[:, 0], a.shape[0]).reshape((-1,1)), np.repeat(a[:, 1], a.shape[0]).reshape((-1,1)), np.tile(a, (a.shape[0], 1))], axis=1), columns=['artist1', 'image1', 'artist2', 'image2'])
		a2 = a2.loc[0:nm, :]
		b2 = pd.DataFrame(np.concatenate([np.tile(a, (a.shape[0], 1))[0:b.shape[0], :], b], axis=1), columns=['artist1', 'image1', 'artist2', 'image2'])
		#print j, i, a2.shape[0], b2.shape[0]
		#print b2
		t.iloc[i:i+a2.shape[0], :] = a2.values
		t.iloc[i+a2.shape[0]:i+a2.shape[0]+b2.shape[0], :] = b2.values
		i += a2.shape[0] +b2.shape[0]
		j += 1
	
	t = t[~t.image2.isin([np.nan, 0])]	
	#return t[t.image1 > t.image2]	
	return t.drop_duplicates(subset=['artist1', 'artist2','image1', 'image2'], keep=False)

	

In [9]:
#x_train, y_train, x_cv, y_cv = prep_data([train_info, None], 'cv')	
#x_test, y_test = prep_data([None, submission_info], 'test')	
def prep_data(input, split):
	info = input[0]
	data = input[1]
	
	if split=='cv':
		#artists = info.artist.unique()
		artists = info.artist
		#print artists
		#print 'hi', artists
		np.random.shuffle(artists)
		
		info = get_image_info(info, 'train')
		info['bytes_per_pixel'] = 1.0*info['size_bytes']/(info['pixelsx']*info['pixelsy'])
		info['aspect_ratio'] = 1.0*info['pixelsx']/info['pixelsy']	
		#train_artists = artists[0:int(0.8*len(artists))]
		#test_artists = artists[int(0.8*len(artists)):]
		#print artists
		print 'hi',info[info.artist.isin(artists)].shape
		#train = make_pairs(info[info.artist.isin(artists)])
		#test = make_pairs(info[info.artist.isin(test_artists)])
		#print train.shape
		#train['in_train'] = True
		#test['in_train'] = True
		print info.columns
		info['artist'] = info['artist'].map({'hudsonriver': 1, 'impressionist': 0})
		y_train = info['artist']
		x_train = info.drop(['artist', 'new_filename'], axis=1) 
		print x_train.columns
		print y_train
		print x_train
		#data['sameArtist'] = data['artist1'] == data['artist2']
		
	if split=='test':

		info = get_image_info(data, 'test')
		info['bytes_per_pixel'] = 1.0*info['size_bytes']/(info['pixelsx']*info['pixelsy'])
		info['aspect_ratio'] = 1.0*info['pixelsx']/info['pixelsy']	
		
		data['in_train'] = False
	
		if 'artist1' in data.columns:
			data['sameArtist'] = data['artist1'] == data['artist2']

	
	#data2 = pd.merge(data, info[['new_filename', 'pixelsx', 'pixelsy', 'size_bytes', 'bytes_per_pixel', 'aspect_ratio']], how='left', left_on='image1', right_on='new_filename')
	#data2.drop('new_filename', 1, inplace=True)
	
	#data2 = pd.merge(data2, info[['new_filename', 'pixelsx', 'pixelsy', 'size_bytes', 'bytes_per_pixel', 'aspect_ratio']], how='left', left_on='image2', right_on='new_filename')
	#data2.drop('new_filename', 1, inplace=True)
	
	#x_train = data2[data2.in_train==True][['pixelsx_x', 'pixelsy_x', 'size_bytes_x', 'bytes_per_pixel_x', 'aspect_ratio_x', 'pixelsx_y', 'pixelsy_y', 'size_bytes_y', 'bytes_per_pixel_y', 'aspect_ratio_y']].values
	#x_test = data2[data2.in_train==False][['pixelsx_x', 'pixelsy_x', 'size_bytes_x', 'bytes_per_pixel_x', 'aspect_ratio_x', 'pixelsx_y', 'pixelsy_y', 'size_bytes_y', 'bytes_per_pixel_y', 'aspect_ratio_y']].values
	
	
# 	if 'artist1' in data.columns: 
# 		y_train = data2[data2.in_train==True]['sameArtist'].values
# 		y_test = data2[data2.in_train==False]['sameArtist'].values
# 	else:
# 		y_test = None	
	
	if split=='cv':		
		return x_train, y_train, x_train, y_train  
 	if split=='test':
		return x_test, y_test

In [ ]:
import pandas as pd
import numpy as np
import time

start_time = time.time()
train_info = pd.read_csv('/Users/amikar/Downloads/ImageSets/pilot_medium.csv')
#submission_info = pd.read_csv('submission_info.csv')
print 'prepping training and cv data'
x_train, y_train, x_cv, y_cv = prep_data([train_info, None], 'cv')

print x_train.shape
#np.savetxt('x_train_pilot.txt', x_train, fmt = '%1.3f' )
#np.savetxt('y_train_pilot.txt', y_train, fmt = '%1.3f' )
#x_train.to_csv("pd_x_train_pilot.txt")
#y_train.to_csv("pd_y_train_pilot.txt")

print (time.time() - start_time)/60 , "minutes"

#print 'prepping test data'
#x_test, y_test = prep_data([None, submission_info], 'test')

prepping training and cv data
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Bierstadt-1863.jpg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Bierstadt-1865-7.jpg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Bierstadt-1869.jpg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Bierstadt-1876-7.jpg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Bierstadt-1895.jpg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Church-1849.jpg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Church-1851.jpeg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Church-1855.jpg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Church-1857.jpg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Church-1859.jpg
 /Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Cole-1827a.jpg
/Users/amikar/Downloads/ImageSets/Pilot/Artwork_for_Testing/Cole-1827b.jpg
/Users/amikar/Downloads/ImageSets/Pilot